In [ ]:
! pip install tabula-py camelot-py opencv-python pandas

In [ ]:
! pip install pytesseract pdf2image PyPDF2

In [ ]:
import os
import tabula
import pandas as pd
import camelot

In [ ]:
! pip install crewai

In [9]:
import os
import pdfplumber
import pandas as pd
import re

def extract_pdf_text(pdf_path: str, output_dir: str = "extracted_text", start_page: int = None, end_page: int = None):
    """
    Extract text from specified pages of a PDF file and save it to a single file.
    Uses pdfplumber for better text extraction and preserves article formatting.
    
    Args:
        pdf_path: Path to the PDF file
        output_dir: Directory to save extracted text
        start_page: First page to extract (starting from 1)
        end_page: Last page to extract
    """
    if not os.path.exists(pdf_path):
        print(f"Erreur: Fichier non trouvé: {pdf_path}")
        return
    
    # Créer le répertoire de sortie s'il n'existe pas
    os.makedirs(output_dir, exist_ok=True)
    
    # Déterminer la plage de pages
    if start_page and end_page:
        page_info = f" (pages {start_page}-{end_page})"
    elif start_page:
        page_info = f" (à partir de la page {start_page})"
    else:
        page_info = " (toutes les pages)"
        start_page = 1
        
    if not end_page:
        end_page = float('inf')  # Sans limite
    
    print(f"Extraction du texte de {pdf_path}{page_info}...")
    
    try:
        # Ouvrir le PDF avec pdfplumber au lieu de PyPDF2
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            
            # Ajuster la plage de pages si nécessaire
            start_idx = max(0, start_page - 1)  # PDF pages commencent à 0 en Python
            end_idx = min(total_pages, end_page)
            
            print(f"Le PDF contient {total_pages} pages. Extraction des pages {start_idx+1} à {end_idx}.")
            
            # Préparer un seul fichier pour toutes les pages
            pdf_name = os.path.basename(pdf_path).replace(".pdf", "")
            output_filename = f"{pdf_name}_pages_{start_page}-{end_idx}.txt"
            output_path = os.path.join(output_dir, output_filename)
            
            # Extraire et écrire le texte de toutes les pages dans un seul fichier
            with open(output_path, 'w', encoding='utf-8') as output_file:
                for i in range(start_idx, end_idx):
                    page = pdf.pages[i]
                    page_text = page.extract_text()
                    
                    # Post-traitement pour corriger les problèmes de formatage
                    page_text = post_process_text(page_text)
                    
                    # Amélioration du formatage des articles
                    page_text = format_articles(page_text)
                    
                    # Écrire un séparateur de page clair
                    output_file.write(f"\n\n{'='*20} PAGE {i+1} {'='*20}\n\n")
                    output_file.write(page_text)
                    output_file.write("\n")
            
            print(f"Texte extrait et sauvegardé dans un seul fichier: {output_path}")
            
    except Exception as e:
        print(f"Erreur lors de l'extraction du texte: {str(e)}")
    
    print("\nExtraction terminée. Vérifiez le fichier", output_path)
    return output_path

def post_process_text(text):
    """
    Fonction pour nettoyer et reformater le texte extrait.
    Corrige les problèmes courants comme les articles mal formatés.
    """
    # Corriger "A\nrticle" en "Article"
    text = re.sub(r'A\s*\n\s*rticle\s+(\d+)', r'Article \1', text)
    
    # Corriger "E\nn cas" en "En cas"
    text = re.sub(r'E\s*\n\s*n cas', r'En cas', text)
    
    # Corriger les sauts de lignes intempestifs dans le texte courant
    # Mais préserver les sauts de ligne pour les éléments spéciaux
    text = re.sub(r'(\w)\s*\n\s*(\w)', lambda m: f"{m.group(1)} {m.group(2)}" if not (m.group(2).lower() == 'les' or m.group(1) == '-') else f"{m.group(1)}\n{m.group(2)}", text)
    
    return text

def format_articles(text):
    """
    Améliore le formatage des articles pour respecter la mise en page désirée:
    - Chaque "Article X" sur sa propre ligne
    - Contenu formaté avec des sauts de ligne appropriés
    - Points avec tirets sur des lignes séparées
    """
    # Assurer que chaque "Article X" est sur sa propre ligne
    text = re.sub(r'(Article \d+[^\n]*)', r'\n\1\n', text)
    
    # Assurer que chaque élément commençant par un tiret est sur sa propre ligne
    text = re.sub(r'([^\n])(\s*-\s+)', r'\1\n\2', text)
    
    # Assurer que "dans la limite de ses attributions et fonctions :" est sur sa propre ligne
    text = re.sub(r'(dans la limite de ses attributions et fonctions\s*:)', r'\n\1\n', text)
    
    # Assurer que "Direction" commence sur une nouvelle ligne
    text = re.sub(r'([^\n])(Direction)', r'\1\n\n\2', text)
    
    # Assurer un saut de ligne avant "En cas d'absence"
    text = re.sub(r'([^\n])(En cas d\'absence)', r'\1\n\2', text)
    
    # Assurer que "délégation est donnée" commence sur une nouvelle ligne s'il n'est pas précédé par "Article"
    text = re.sub(r'([^\n])(délégation est donnée)', lambda m: f"{m.group(1)}\n{m.group(2)}" if "Article" not in m.group(1) else f"{m.group(1)} {m.group(2)}", text)
    
    # Nettoyer les lignes vides multiples
    text = re.sub(r'\n{3,}', r'\n\n', text)
    
    return text

# Fonction pour afficher un aperçu du texte extrait (inchangée)
def preview_extracted_text(file_path):
    """Affiche un aperçu du fichier texte extrait"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            
            print("\n" + "="*50)
            print(f"Fichier: {os.path.basename(file_path)}")
            print(f"Taille: {len(content)} caractères")
            print("-"*50)
            
            # Trouver les 2 premières pages pour l'aperçu
            parts = content.split("=" * 20 + " PAGE ")
            
            if len(parts) > 1:
                # Afficher un extrait de la première page
                first_page = parts[1].split("=" * 20, 1)[1] if len(parts[1].split("=" * 20, 1)) > 1 else parts[1]
                preview = first_page[:300] + "..." if len(first_page) > 300 else first_page
                print("Aperçu de la première page:")
                print(preview)
                
                # Afficher le nombre total de pages extraites
                print(f"\nNombre total de pages extraites: {len(parts)-1}")
            else:
                # Fallback si le format n'est pas comme attendu
                preview = content[:300] + "..." if len(content) > 300 else content
                print(preview)
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier: {str(e)}")

# Pour traiter tous les fichiers dans la liste
pdf_paths = [
    r"C:\Users\kosmo\pycode\MCP_Rag_virg\2025.5.sante.pdf"
    # Ajoutez d'autres chemins si nécessaire
]

output_dir = "textes_extraits"

# Parcourir chaque PDF dans la liste
for i, pdf_path in enumerate(pdf_paths):
    print(f"\nTraitement du PDF {i+1}/{len(pdf_paths)}: {pdf_path}")
    
    # Extraire le texte - ici vous pouvez spécifier les pages (125 à 132 par exemple)
    output_file = extract_pdf_text(pdf_path, output_dir, start_page=117, end_page=131)
    
    # Afficher un aperçu
    preview_extracted_text(output_file)


Traitement du PDF 1/1: C:\Users\kosmo\pycode\MCP_Rag_virg\2025.5.sante.pdf
Extraction du texte de C:\Users\kosmo\pycode\MCP_Rag_virg\2025.5.sante.pdf (pages 117-131)...
Le PDF contient 146 pages. Extraction des pages 117 à 131.
Texte extrait et sauvegardé dans un seul fichier: textes_extraits\2025.5.sante_pages_117-131.txt

Extraction terminée. Vérifiez le fichier textes_extraits\2025.5.sante_pages_117-131.txt

Fichier: 2025.5.sante_pages_117-131.txt
Taille: 41427 caractères
--------------------------------------------------
Aperçu de la première page:


Bulletin officiel Santé
 - Protection sociale
 - Solidarité n° 2025/5 du 17 mars 2025 Page 117 Agence nationale de santé publique / Santé publique France Décision DG n° 71-2025 du 12 mars 2025 portant délégation de signature au sein de Santé publique France, l’Agence nationale de santé publique NO...

Nombre total de pages extraites: 15


In [12]:
import os
import pdfplumber
import pandas as pd
import re

def extract_pdf_text(pdf_path: str, output_dir: str = "extracted_text", start_page: int = None, end_page: int = None):
    """
    Extract text from specified pages of a PDF file and save it to a single file.
    Uses pdfplumber for better text extraction and preserves article formatting.
    
    Args:
        pdf_path: Path to the PDF file
        output_dir: Directory to save extracted text
        start_page: First page to extract (starting from 1)
        end_page: Last page to extract
    """
    if not os.path.exists(pdf_path):
        print(f"Erreur: Fichier non trouvé: {pdf_path}")
        return
    
    # Créer le répertoire de sortie s'il n'existe pas
    os.makedirs(output_dir, exist_ok=True)
    
    # Déterminer la plage de pages
    if start_page and end_page:
        page_info = f" (pages {start_page}-{end_page})"
    elif start_page:
        page_info = f" (à partir de la page {start_page})"
    else:
        page_info = " (toutes les pages)"
        start_page = 1
        
    if not end_page:
        end_page = float('inf')  # Sans limite
    
    print(f"Extraction du texte de {pdf_path}{page_info}...")
    
    try:
        # Ouvrir le PDF avec pdfplumber au lieu de PyPDF2
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            
            # Ajuster la plage de pages si nécessaire
            start_idx = max(0, start_page - 1)  # PDF pages commencent à 0 en Python
            end_idx = min(total_pages, end_page)
            
            print(f"Le PDF contient {total_pages} pages. Extraction des pages {start_idx+1} à {end_idx}.")
            
            # Préparer un seul fichier pour toutes les pages
            pdf_name = os.path.basename(pdf_path).replace(".pdf", "")
            output_filename = f"{pdf_name}_pages_{start_page}-{end_idx}.txt"
            output_path = os.path.join(output_dir, output_filename)
            
            # Extraire et écrire le texte de toutes les pages dans un seul fichier
            with open(output_path, 'w', encoding='utf-8') as output_file:
                for i in range(start_idx, end_idx):
                    page = pdf.pages[i]
                    page_text = page.extract_text()
                    
                    # Post-traitement pour corriger les problèmes de formatage
                    page_text = post_process_text(page_text)
                    
                    # Amélioration du formatage des articles
                    page_text = format_articles(page_text)
                    
                    # Écrire un séparateur de page clair
                    output_file.write(f"\n\n{'='*20} PAGE {i+1} {'='*20}\n\n")
                    output_file.write(page_text)
                    output_file.write("\n")
            
            print(f"Texte extrait et sauvegardé dans un seul fichier: {output_path}")
            
    except Exception as e:
        print(f"Erreur lors de l'extraction du texte: {str(e)}")
    
    print("\nExtraction terminée. Vérifiez le fichier", output_path)
    return output_path

def post_process_text(text):
    """
    Fonction pour nettoyer et reformater le texte extrait.
    Corrige les problèmes courants comme les articles mal formatés.
    """
    # Corriger "A\nrticle" en "Article"
    text = re.sub(r'A\s*\n\s*rticle\s+(\d+)', r'Article \1', text)
    
    # Corriger "E\nn cas" en "En cas"
    text = re.sub(r'E\s*\n\s*n cas', r'En cas', text)
    
    # Corriger les sauts de lignes intempestifs dans le texte courant
    # Mais préserver les sauts de ligne pour les éléments spéciaux
    text = re.sub(r'(\w)\s*\n\s*(\w)', lambda m: f"{m.group(1)} {m.group(2)}" if not (m.group(2).lower() == 'les' or m.group(1) == '-') else f"{m.group(1)}\n{m.group(2)}", text)
    
    return text

def format_articles(text):
    """
    Améliore le formatage des articles pour respecter la mise en page désirée:
    - Chaque "Article X" sur sa propre ligne suivie d'un saut de ligne
    - Contenu formaté avec des sauts de ligne appropriés
    - Points avec tirets sur des lignes séparées
    """
    # CORRECTION IMPORTANTE: Traiter les articles d'une manière qui préserve le texte de délégation
    # Mettre "Article XX" sur une ligne séparée et ajouter un saut de ligne après
    text = re.sub(r'\b(Article\s+\d+)\s+', r'\n\1\n\n', text)
    
    # Assurer que "Direction de" commence toujours sur une nouvelle ligne
    text = re.sub(r'([^\n])(Direction de)', r'\1\n\n\2', text)
    
    # Assurer que chaque élément commençant par un tiret est sur une nouvelle ligne
    text = re.sub(r'([^\n])\s*(-\s+)', r'\1\n\2', text)
    
    # Assurer que "dans la limite de ses attributions et fonctions :" est sur sa propre ligne
    text = re.sub(r'(dans la limite de ses attributions et fonctions\s*:)', r'\n\1\n', text)
    
    # S'assurer que le texte après un tiret soit bien séparé de la Direction qui suit
    text = re.sub(r'(afférents\.)\s*(Direction)', r'\1\n\n\2', text)
    text = re.sub(r'(comptes\.)\s*(Direction)', r'\1\n\n\2', text)
    
    # S'assurer que "Bulletin officiel" est au début d'une ligne
    text = re.sub(r'([^\n])(Bulletin officiel)', r'\1\n\2', text)
    
    # Nettoyer les lignes vides multiples (pas plus de 2 consécutives)
    text = re.sub(r'\n{3,}', r'\n\n', text)
    
    return text

# Fonction pour afficher un aperçu du texte extrait (inchangée)
def preview_extracted_text(file_path):
    """Affiche un aperçu du fichier texte extrait"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            
            print("\n" + "="*50)
            print(f"Fichier: {os.path.basename(file_path)}")
            print(f"Taille: {len(content)} caractères")
            print("-"*50)
            
            # Trouver les 2 premières pages pour l'aperçu
            parts = content.split("=" * 20 + " PAGE ")
            
            if len(parts) > 1:
                # Afficher un extrait de la première page
                first_page = parts[1].split("=" * 20, 1)[1] if len(parts[1].split("=" * 20, 1)) > 1 else parts[1]
                preview = first_page[:300] + "..." if len(first_page) > 300 else first_page
                print("Aperçu de la première page:")
                print(preview)
                
                # Afficher le nombre total de pages extraites
                print(f"\nNombre total de pages extraites: {len(parts)-1}")
            else:
                # Fallback si le format n'est pas comme attendu
                preview = content[:300] + "..." if len(content) > 300 else content
                print(preview)
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier: {str(e)}")

# Pour traiter tous les fichiers dans la liste
pdf_paths = [
    r"C:\Users\kosmo\pycode\MCP_Rag_virg\2025.5.sante.pdf"
    # Ajoutez d'autres chemins si nécessaire
]

output_dir = "textes_extraits"

# Parcourir chaque PDF dans la liste
for i, pdf_path in enumerate(pdf_paths):
    print(f"\nTraitement du PDF {i+1}/{len(pdf_paths)}: {pdf_path}")
    
    # Extraire le texte - ici vous pouvez spécifier les pages (125 à 132 par exemple)
    output_file = extract_pdf_text(pdf_path, output_dir, start_page=117, end_page=131)
    
    # Afficher un aperçu
    preview_extracted_text(output_file)


Traitement du PDF 1/1: C:\Users\kosmo\pycode\MCP_Rag_virg\2025.5.sante.pdf
Extraction du texte de C:\Users\kosmo\pycode\MCP_Rag_virg\2025.5.sante.pdf (pages 117-131)...
Le PDF contient 146 pages. Extraction des pages 117 à 131.
Texte extrait et sauvegardé dans un seul fichier: textes_extraits\2025.5.sante_pages_117-131.txt

Extraction terminée. Vérifiez le fichier textes_extraits\2025.5.sante_pages_117-131.txt

Fichier: 2025.5.sante_pages_117-131.txt
Taille: 41302 caractères
--------------------------------------------------
Aperçu de la première page:


Bulletin officiel Santé
- Protection sociale
- Solidarité n° 2025/5 du 17 mars 2025 Page 117 Agence nationale de santé publique / Santé publique France Décision DG n° 71-2025 du 12 mars 2025 portant délégation de signature au sein de Santé publique France, l’Agence nationale de santé publique NOR ...

Nombre total de pages extraites: 15
